In [28]:
import pandas as pd

# Create the Patients DataFrame
patients = pd.DataFrame({
    'patient_id': [1, 2, 3, 4, 5],
    'patient_name': ['Alice Smith', 'Bob Johnson', 'Carol Davis', 'David Wilson', 'Emma Brown'],
    'age': [28, 35, 42, 31, 29]
})

# Display the DataFrame
print(patients)


   patient_id  patient_name  age
0           1   Alice Smith   28
1           2   Bob Johnson   35
2           3   Carol Davis   42
3           4  David Wilson   31
4           5    Emma Brown   29


In [16]:
import pandas as pd

# Create the Covid Tests DataFrame
covid_tests = pd.DataFrame({
    'test_id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
    'patient_id': [1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 5, 5],
    'test_date': [
        '2023-01-15', '2023-01-25', '2023-02-01', '2023-02-05', '2023-02-12',
        '2023-01-20', '2023-02-10', '2023-02-20', '2023-01-10', '2023-01-18',
        '2023-02-15', '2023-02-20'
    ],
    'result': [
        'Positive', 'Negative', 'Positive', 'Inconclusive', 'Negative',
        'Negative', 'Positive', 'Negative', 'Positive', 'Positive',
        'Negative', 'Negative'
    ]
})

# Convert test_date column to datetime for better analysis
covid_tests['test_date'] = pd.to_datetime(covid_tests['test_date'])

# Display the DataFrame
print(covid_tests)


    test_id  patient_id  test_date        result
0         1           1 2023-01-15      Positive
1         2           1 2023-01-25      Negative
2         3           2 2023-02-01      Positive
3         4           2 2023-02-05  Inconclusive
4         5           2 2023-02-12      Negative
5         6           3 2023-01-20      Negative
6         7           3 2023-02-10      Positive
7         8           3 2023-02-20      Negative
8         9           4 2023-01-10      Positive
9        10           4 2023-01-18      Positive
10       11           5 2023-02-15      Negative
11       12           5 2023-02-20      Negative


In [17]:
first_positive1=covid_tests[covid_tests['result']=='Positive'].groupby('patient_id')['test_date'].min().reset_index(name='first_positive')
first_positive1

,patient_id,first_positive
0,1,2023-01-15
1,2,2023-02-01
2,3,2023-02-10
3,4,2023-01-10


In [18]:
merged_df=first_positive1.merge(covid_tests,how='inner',on='patient_id')
merged_df

,patient_id,first_positive,test_id,test_date,result
0,1,2023-01-15,1,2023-01-15,Positive
1,1,2023-01-15,2,2023-01-25,Negative
2,2,2023-02-01,3,2023-02-01,Positive
3,2,2023-02-01,4,2023-02-05,Inconclusive
4,2,2023-02-01,5,2023-02-12,Negative
5,3,2023-02-10,6,2023-01-20,Negative
6,3,2023-02-10,7,2023-02-10,Positive
7,3,2023-02-10,8,2023-02-20,Negative
8,4,2023-01-10,9,2023-01-10,Positive
9,4,2023-01-10,10,2023-01-18,Positive


In [ ]:
filtered_negative=merged_df[(merged_df['result']=='Negative') & (merged_df['test_date']>merged_df['first_positive'])].groupby('patient_id')['test_date'].transform('min')
filtered_negative

,patient_id,first_positive,test_id,test_date,result
1,1,2023-01-15,2,2023-01-25,Negative
4,2,2023-02-01,5,2023-02-12,Negative
7,3,2023-02-10,8,2023-02-20,Negative


In [24]:
merged_df['first_negative']=merged_df[(merged_df['result']=='Negative') & (merged_df['test_date']>merged_df['first_positive'])].groupby('patient_id')['test_date'].transform('min')
merged_df

,patient_id,first_positive,test_id,test_date,result,first_negative
0,1,2023-01-15,1,2023-01-15,Positive,NaT
1,1,2023-01-15,2,2023-01-25,Negative,2023-01-25
2,2,2023-02-01,3,2023-02-01,Positive,NaT
3,2,2023-02-01,4,2023-02-05,Inconclusive,NaT
4,2,2023-02-01,5,2023-02-12,Negative,2023-02-12
5,3,2023-02-10,6,2023-01-20,Negative,NaT
6,3,2023-02-10,7,2023-02-10,Positive,NaT
7,3,2023-02-10,8,2023-02-20,Negative,2023-02-20
8,4,2023-01-10,9,2023-01-10,Positive,NaT
9,4,2023-01-10,10,2023-01-18,Positive,NaT


In [39]:
filtered_df=merged_df[merged_df['first_negative'].notnull()]
filtered_df=filtered_df.copy()

In [40]:
filtered_df.loc[:,'recovery_time']=(filtered_df['first_negative']-filtered_df['first_positive']).dt.days
filtered_df

,patient_id,first_positive,test_id,test_date,result,first_negative,recovery_time
1,1,2023-01-15,2,2023-01-25,Negative,2023-01-25,10
4,2,2023-02-01,5,2023-02-12,Negative,2023-02-12,11
7,3,2023-02-10,8,2023-02-20,Negative,2023-02-20,10


In [32]:
patient_info=filtered_df.merge(patients,on='patient_id',how='inner')
patient_info

,patient_id,first_positive,test_id,test_date,result,first_negative,recovery_time,patient_name,age
0,1,2023-01-15,2,2023-01-25,Negative,2023-01-25,10,Alice Smith,28
1,2,2023-02-01,5,2023-02-12,Negative,2023-02-12,11,Bob Johnson,35
2,3,2023-02-10,8,2023-02-20,Negative,2023-02-20,10,Carol Davis,42


In [42]:
patient_info[['patient_id','patient_name','age','recovery_time']].sort_values('recovery_time',ascending=True)

,patient_id,patient_name,age,recovery_time
0,1,Alice Smith,28,10
2,3,Carol Davis,42,10
1,2,Bob Johnson,35,11
